In [1]:
import os
import cv2
import numpy as np
import json
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
DATA_PATH = os.path.join(os.getcwd(), '../data/8_pts_dense')
print(DATA_PATH)
PROCESSED_DATA_PATH = os.path.join(os.getcwd(), '../processed/8_pts_dense')
if not os.path.exists(PROCESSED_DATA_PATH):
    os.makedirs(PROCESSED_DATA_PATH)
if not os.path.exists(os.path.join(PROCESSED_DATA_PATH,'images')):
    os.makedirs(os.path.join(PROCESSED_DATA_PATH,'images'))

/home/boomalope/nerf_estimator/src/nerf_estimator/nerf_vision/scripts/../data/8_pts_dense


In [3]:
#Gazebo Param: 
#Definition: https://github.com/ros-simulation/gazebo_ros_pkgs/blob/398cde933cba5d72921055d57d4faa47c4445654/gazebo_plugins/src/gazebo_ros_camera_utils.cpp#L524

In [4]:
#Camera parameters
#Definitions: https://docs.nerf.studio/quickstart/data_conventions.html

fl_x = 762.72 # focal length in x
fl_y = 762.72# focal length in y
w = 1280 # image width
h = 720 # image height
cx = w//2 # optical center in x
cy = h//2 # optical center in y
camera_model = "OPENCV" # camera model
k1 = 0 # radial distortion coefficient k1
k2 = 0 # radial distortion coefficient k2
k3 = 0 # radial distortion coefficient k3
p1 = 0 # tangential distortion coefficient p1
p2 = 0 # tangential distortion coefficient p2

In [5]:
def from_tf_to_matrix(tf):
    #tf is the frame of the robot relative to the camera
    xyz = tf[:3]
    q = tf[3:7]
    goal = tf[7:]

    #Rotation matrix
    r = Rotation.from_quat(q)
    r = r.as_matrix()

    #Translation matrix
    t = np.zeros((4,4))
    t[:3,:3] = r.T
    t[:3,3] = -np.dot(r.T, xyz)
    t[3,3] = 1

    #Tranform according to nerfstudio coordinates
    R = np.eye(4)
    R[1,1] = -1
    R[2,2] = -1
    t = np.dot(R,t)

    return t

In [6]:
data_cnt = 0
frames = []
while True:
    #Get data paths
    data_img = os.path.join(DATA_PATH, '{}.png'.format(data_cnt))
    data_tf= os.path.join(DATA_PATH, '{}.txt'.format(data_cnt))

    if not os.path.exists(data_img):
        break
    
    #Get tf as np array
    tf = np.loadtxt(data_tf, delimiter=',')


    file_path = "images/{}.png".format(data_cnt)
    transform_matrix = from_tf_to_matrix(tf)

    #Append to frames
    frames.append({
        "file_path": file_path,
        "transform_matrix": transform_matrix.tolist(),
    })

    #Save image in processed folder
    img = cv2.imread(data_img)
    cv2.imwrite(os.path.join(PROCESSED_DATA_PATH, file_path), img)
    
    data_cnt += 1        

[[ 9.19641780e-01 -2.77729827e-01  2.77714133e-01 -2.10653169e+00]
 [-3.92758189e-01 -6.50363258e-01  6.50206612e-01  1.54082194e+00]
 [ 3.32983871e-05 -7.07031666e-01 -7.07181888e-01 -2.00025410e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.19270925e-01 -2.78340446e-01  2.78329952e-01 -2.10463783e+00]
 [-3.93625411e-01 -6.50117345e-01  6.49928052e-01  1.54277032e+00]
 [ 4.58657626e-05 -7.07017704e-01 -7.07195846e-01 -2.00029768e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.87222071e-01 -1.12687101e-01  1.12668543e-01 -2.41580260e+00]
 [-1.59350504e-01 -6.98137486e-01  6.98005349e-01  9.87368882e-01]
 [ 1.93418063e-06 -7.07040075e-01 -7.07173481e-01 -2.00016734e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.97006015e-01  5.46737225e-02 -5.46789793e-02 -2.58033344e+00]
 [ 7.73240363e-02 -7.05059870e-01  7.04919551e-01  3.91467053e-01]
 [-1.13781110e-05 -7.07037032e-01 -7.07176524e-01 -2.000136

[[ 9.46147505e-01  2.28901645e-01 -2.28929979e-01 -2.59727959e+00]
 [ 3.23735841e-01 -6.69112676e-01  6.68941950e-01 -2.65122984e-01]
 [-5.80377749e-05 -7.07030597e-01 -7.07182955e-01 -2.00002278e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 8.19457301e-01  4.05275845e-01 -4.05266852e-01 -2.42243931e+00]
 [ 5.73140235e-01 -5.79519245e-01  5.79368376e-01 -9.74173567e-01]
 [-5.59319748e-05 -7.07042384e-01 -7.07171170e-01 -2.00001831e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 6.57144039e-01  5.32981790e-01 -5.33002929e-01 -2.11634895e+00]
 [ 7.53765021e-01 -4.64728801e-01  4.64613210e-01 -1.52926783e+00]
 [-7.14318057e-05 -7.07076765e-01 -7.07136792e-01 -1.99995020e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 4.47149337e-01  6.32472718e-01 -6.32483779e-01 -1.66636443e+00]
 [ 8.94459312e-01 -3.16258482e-01  3.16106173e-01 -2.01134668e+00]
 [-9.98292240e-05 -7.07077671e-01 -7.07135883e-01 -1.999877

In [ ]:
transform_dictionary = {}
transform_dictionary["fl_x"] = fl_x
transform_dictionary["fl_y"] = fl_y
transform_dictionary["w"] = w
transform_dictionary["h"] = h
transform_dictionary["cx"] = cx
transform_dictionary["cy"] = cy
transform_dictionary["k1"] = k1
transform_dictionary["k2"] = k2
transform_dictionary["k3"] = k3
transform_dictionary["p1"] = p1
transform_dictionary["p2"] = p2
transform_dictionary["camera_model"] = camera_model
transform_dictionary["frames"] = frames

#Save as json 
with open(os.path.join(PROCESSED_DATA_PATH, "transforms.json"), 'w') as outfile:
    json.dump(transform_dictionary, outfile)
